In [8]:
import os
import csv

# Defining the paths
base_dir = 'C:\\Users\\admin\\OneDrive\\Desktop\\ML projects\\Engine defects\\archive (1)\\Aero-engine_defect-detect_new'
images_dir = os.path.join(base_dir, 'images')
labels_dir = os.path.join(base_dir, 'labels')

# loading the CSV file
csv_file = os.path.join(base_dir, 'defect_data.csv')

# Function to process a directory
def process_directory(image_subdir, label_subdir, csv_writer):
    image_folder = os.path.join(images_dir, image_subdir)
    label_folder = os.path.join(labels_dir, label_subdir)

    for image_filename in os.listdir(image_folder):
        if image_filename.endswith('.jpg') or image_filename.endswith('.png'):
            image_path = os.path.join(image_folder, image_filename)
            label_filename = image_filename.replace('.jpg', '.txt').replace('.png', '.txt')
            label_path = os.path.join(label_folder, label_filename)

            if os.path.exists(label_path):
                with open(label_path, 'r') as label_file:
                    for line in label_file:
                        parts = line.strip().split()
                        defect_type = parts[0]
                        x1, y1, x2, y2 = parts[1:]
                        csv_writer.writerow([image_path, defect_type, x1, y1, x2, y2])

# Creating the CSV file
with open(csv_file, 'w', newline='') as csvfile:
    csv_writer = csv.writer(csvfile)
    csv_writer.writerow(['image_path', 'defect_type', 'x1', 'y1', 'x2', 'y2'])

    # Process train and val directories
    process_directory('train', 'train', csv_writer)
    process_directory('val', 'val', csv_writer)

print(f"CSV file created at {csv_file}")


CSV file created at C:\Users\admin\OneDrive\Desktop\ML projects\Engine defects\archive (1)\Aero-engine_defect-detect_new\defect_data.csv


In [10]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
import cv2
import os

# Loading the CSV file
csv_file = 'C:\\Users\\admin\\OneDrive\\Desktop\\ML projects\\Engine defects\\archive (1)\\Aero-engine_defect-detect_new\\defect_data.csv'
data = pd.read_csv(csv_file)

# Function to load and preprocess images
def load_images(data, image_size=(128, 128)):
    images = []
    labels = []
    for index, row in data.iterrows():
        image_path = row['image_path']
        defect_type = row['defect_type']
        image = cv2.imread(image_path)
        image = cv2.resize(image, image_size)
        images.append(image)
        labels.append(defect_type)
    images = np.array(images)
    labels = np.array(labels).astype('float32')
    return images, labels

# Load and preprocess images and labels
image_size = (128, 128)

# Splitting the data into training and validation sets
train_data = data[data['image_path'].str.contains('train')]
val_data = data[data['image_path'].str.contains('val')]
X_train, y_train = load_images(train_data, image_size=image_size)
X_val, y_val = load_images(val_data, image_size=image_size)

# Normalizing pixel values to between 0 and 1
X_train = X_train / 255.0
X_val = X_val / 255.0

# Converting labels to one-hot encoding
y_train = tf.keras.utils.to_categorical(y_train, num_classes=4)
y_val = tf.keras.utils.to_categorical(y_val, num_classes=4)

# Creating the CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(image_size[0], image_size[1], 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='softmax')
])

# Compiling the model
model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])

# Training the model
#early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val))

# Evaluating the model
val_loss, val_accuracy = model.evaluate(X_val, y_val)
print(f'Validation Loss: {val_loss}')
print(f'Validation Accuracy: {val_accuracy}')

# Save the model
model.save('defect_detection_model.h5')



C:\Users\admin\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 68s 2s/step - accuracy: 0.3484 - loss: 1.4563 - val_accuracy: 0.6579 - val_loss: 0.9650
Epoch 2/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.5553 - loss: 1.0658 - val_accuracy: 0.6184 - val_loss: 0.9691
Epoch 3/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 40s 2s/step - accuracy: 0.5827 - loss: 0.9329 - val_accuracy: 0.6053 - val_loss: 0.9312
Epoch 4/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.5968 - loss: 0.8900 - val_accuracy: 0.6316 - val_loss: 0.8785
Epoch 5/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.6295 - loss: 0.8624 - val_accuracy: 0.6711 - val_loss: 1.1049
Epoch 6/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.6945 - loss: 0.7466 - val_accuracy: 0.6711 - val_loss: 1.4721
Epoch 7/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 26s 2s/step - accuracy: 0.7077 - loss: 0.6302 - val_accuracy: 0.6842 - val_loss: 1.4161
Epoch 8/20
12/12 ━━━━━━━━━━━━━━━━━━━━ 25s 2s/step - accuracy: 0.7660 - loss: 0.5207 - val_accuracy: 0.7237 - val_loss:

Validation Loss: 3.5216310024261475
Validation Accuracy: 0.8026315569877625
